<h1 align='center'>Diferenciación Automática con Tensorflow 2.1</h1> 

<h3>Autor</h3>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 

<h3>Fork</h3>

<h3>Referencias</h3>



<h2>1. Introducción </h2>

[La diferenciación automática](https://en.wikipedia.org/wiki/Automatic_differentiation)
es una técnica clave para la optimización de modelos de aprendizaje de máquinas (machine learning). En este cuaderno se hace una breve descripción de <a href="https://www.tensorflow.org/api_docs/python/tf/GradientTape">tf.GradientTape</a> la API para diferenicación automática en tensorflow 2.1.

<h2>2. Gradient Tapes </h2>

Lo que hace el *administrador de contexto* **tf.GradientTape** es calcular el gradiente de un cálculo con respecto a sus variables de entrada. Tensorflow registra todas las operaciones ejecutadas dentro del contexto de un tf.GradientTape en una cinta. Tensorflow usa esa cinta y los gradientes asociados con cada operación registrada para calcular los gradientes de un cálculo registrado usando el [modo de diferenciación hacia automática atrás](https://en.wikipedia.org/wiki/Automatic_differentiation)


Las operaciones se registran si se ejecutan dentro de este administrador de contexto y al menos una de sus entradas está siendo *observada* (watched). Si una variable es creada con *tf.Variable*  es marcada como entrenable, será observada (registrada).

Por ejemplo:

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
print("Versión de Tensorflow: ", tf.__version__)

Versión de Tensorflow:  2.0.0


In [4]:
x = tf.ones((2,2))

# this is the GradientTape context
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)
    
# Derivate of z with respect to the original input tensor x
dz_dx = t.gradient(z,x)
for i in [0,1]:
    for j in [0,1]:
        assert dz_dx[i][j].numpy() == 8.0


Note that matemáticamente lo que hicimos es lo siguiente:

$$
\begin{equation}
x = \begin{pmatrix} 1 & 1\\1 & 1 \end{pmatrix}
\end{equation}
$$

$$
\begin{align}
y &= x_{11} + x_{12} + x_{21} + x_{22} = 4\\
z &= y^2
\end{align}
$$

La derivada es calculada usando la regla de la cadena.

$$
\begin{equation}
\frac{dz}{dx} = \left( \frac{dz}{dy}\right) \left( \frac{dy}{dx}\right) = 2yx= 8 x = \begin{pmatrix} 8 & 8\\8 & 8 \end{pmatrix}
\end{equation}
$$

In [3]:
dz_dx

<tf.Tensor: id=13, shape=(2, 2), dtype=float32, numpy=
array([[8., 8.],
       [8., 8.]], dtype=float32)>

In [4]:
dz_dx.numpy()

array([[8., 8.],
       [8., 8.]], dtype=float32)

Es posible obtener gradientes de la salida (output) con respecto a valores intermedios  computados en un contexto tf.GradientTape *registrado*. Por defecto, los recursos mantenidos por GradientTape, son liberados tan pronto el método *GradientTape.gradient()* es llamado. Para hacer multiples llamados es necesario crear las instancia de GradientTape en modo persistente. Cuando ya no se requiere más, se recomienda liberar el recurso. Veamos el ejemplo.

In [12]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x * x
    z = y *  y
dz_dx = t.gradient(z,x) # 108.0 (4*x³) evaluate at x=3.0
dy_dx = t.gradient(y,x) #  6.0
del t # delete the reference to the tape ( free the resource)
print(dz_dx.numpy())
print(dy_dx.numpy())

108.0
6.0


<h2>3. Registro del flujo de control  </h2>

Debido a que las cintas (tapes) registran operaciones tal como ellas son ejecutadas, sentencias Python para el control del flujo (por ejemplo *if s* and *while s*) pueden ser manejadas de  manera natural:

In [15]:
def f(x,y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            output = tf.multiply(output,x)
    return output

def grad(x,y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x,y)
    return t.gradient(out,x)

x = tf.convert_to_tensor(2.0)

assert grad(x,6).numpy() == 12.0
assert grad(x,5).numpy() == 12.0
assert grad(x,4).numpy() == 4.0
    

<h2>4. Gradientes de órdenes superiores   </h2>


Las operaciones dentro del administrador de contexto *GradientTape* se registran para la diferenciación automática. Si los gradientes de órdenes superiores  se calculan en ese contexto, el cálculo del gradiente también se registra. Como resultado, la misma API también funciona para gradientes de orden superior. Por ejemplo:

In [16]:
x = tf.Variable(1.0) # Creates a Tensorflow variable intialized to 1.0

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x * x * x
    # Compute the gradient inside the 't' context manager
    # which means the gradient computation is differentiable as well.
    dy_dx = t2.gradient(y,x)
d2y_d2x = t.gradient(dy_dx,x)

assert dy_dx.numpy() == 3.0
assert d2y_d2x.numpy() == 6.0